![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Advanced Futures Trading Strategies

"Advanced Futures Trading Strategies" by Robert Carver lays the foundation for building a systematic trading system.

Although the book focuses on futures trading, its concepts are foundational and applicable to other financial instruments.

This notebook replicates the book's initial strategies, emphasizing data processing, metric building, and position size management with risk targeting.

# Data aquisition

In [1]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from QuantConnect.Data.UniverseSelection import *
from helper import calculate_and_merge_returns, summarize_statistics, calculate_contract_risk, simulate_buy_and_hold, calculate_rolling_costs
import numpy as np
from scipy.stats import skew

In [10]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.SP500EMini,
        'resolution': Resolution.Daily
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.MicroSP500EMini,
        'resolution': Resolution.Daily
    },
    'ZN': {     # 10-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.015625,
        'minimum_fluctuation': 15.625,
        'spread': 0.015625,
        'commission': 2.50,
        'rolling_months': [6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y10TreasuryNote,
        'resolution': Resolution.Daily
    },
    'ZF': {  # 5-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.0078125,
        'minimum_fluctuation': 7.8125,
        'spread': 0.0078125,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y5TreasuryNote,
        'resolution': Resolution.Daily
    },
    'ZB': {  # 30-Year Treasury
        'multiplier': 1000,
        'tick_value': 0.03125,
        'minimum_fluctuation': 31.25,
        'spread': 0.03125,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Financials.Y30TreasuryBond,
        'resolution': Resolution.Daily
    },
    'ZW': {  # Wheat
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.SRWWheat,
        'resolution': Resolution.Daily
    },
    'CL': {  # Crude Oil
        'multiplier': 1000,
        'tick_value': 0.01,
        'minimum_fluctuation': 10.00,
        'spread': 0.01,
        'commission': 2.50,
        'rolling_months': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Energies.CrudeOilWTI,
        'resolution': Resolution.Daily
    },
    'GC': {  # Gold
        'multiplier': 100,
        'tick_value': 0.1,
        'minimum_fluctuation': 10.00,
        'spread': 0.1,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Gold,
        'resolution': Resolution.Daily
    },
    'HE': {  # Lean Hog
        'multiplier': 400,
        'tick_value': 0.00025,
        'minimum_fluctuation': 10.00,
        'spread': 0.025,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 7, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Meats.LeanHogs,
        'resolution': Resolution.Daily
    },
    'GF': {  # Feeder Cattle
        'multiplier': 500,
        'tick_value': 0.00025,
        'minimum_fluctuation': 12.50,
        'spread': 0.00025,
        'commission': 2.50,
        'rolling_months': [1, 3, 4, 5, 8, 9, 10, 11],
        'subscription_type': 'Future',
        'subscription_details': Futures.Meats.FeederCattle,
        'resolution': Resolution.Daily
    },
    'ZS': {  # Soy Bean
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'rolling_months': [1, 3, 5, 7, 8, 9, 11],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.Soybeans,
        'resolution': Resolution.Daily
    },
    'ZC': {  # Corn
        'multiplier': 50,
        'tick_value': 0.0025,
        'minimum_fluctuation': 12.50,
        'spread': 0.0025,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Grains.Corn,
        'resolution': Resolution.Daily
    },
    'NG': {  # Natural Gas
        'multiplier': 10000,
        'tick_value': 0.001,
        'minimum_fluctuation': 10.00,
        'spread': 0.001,
        'commission': 2.50,
        'rolling_months': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Energies.NaturalGas,
        'resolution': Resolution.Daily
    },
    'SI': {  # Silver
        'multiplier': 5000,
        'tick_value': 0.005,
        'minimum_fluctuation': 25.00,
        'spread': 0.005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Silver,
        'resolution': Resolution.Daily
    },
    'HG': {  # Copper
        'multiplier': 25000,
        'tick_value': 0.0005,
        'minimum_fluctuation': 12.50,
        'spread': 0.0005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.Copper,
        'resolution': Resolution.Daily
    },
    'NQ': {  # Nasdaq
        'multiplier': 20,
        'tick_value': 0.25,
        'minimum_fluctuation': 5.00,
        'spread': 0.25,
        'commission': 2.50,
        'rolling_months': [3, 6, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Indices.NASDAQ100EMini,
        'resolution': Resolution.Daily
    },
    'MGC': { # Micro gold
        'multiplier': 10,
        'tick_value': 0.1,
        'minimum_fluctuation': 1.00,
        'spread': 0.1,
        'commission': 2.50,
        'rolling_months': [2, 4, 6, 8, 10, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.MicroGold,
        'resolution': Resolution.Daily
    },
    'SIL': { # Micro silver
        'multiplier': 1000,
        'tick_value': 0.005,
        'minimum_fluctuation': 5.00,
        'spread': 0.005,
        'commission': 2.50,
        'rolling_months': [3, 5, 7, 9, 12],
        'subscription_type': 'Future',
        'subscription_details': Futures.Metals.MicroSilver,
        'resolution': Resolution.Daily
    },
    'USDMXN': { #forex exchange rate to mxn
        'subscription_type': 'Forex',
        'subscription_details': 'USDMXN',
        'resolution': Resolution.Daily,
        'multiplier': 1,
    }
}



In [2]:
# # Define Instruments Dictionary
# instruments = {
#     'ES': {
#         'multiplier': 50,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 12.50,
#         'spread': 0.25,
#         'commission': 2.50,
#         'rolling_months': [3, 6, 9, 12],
#         'subscription_type': 'Future',
#         'subscription_details': Futures.Indices.SP500EMini,
#         'resolution': Resolution.Daily
#     },
#     'MES': {
#         'multiplier': 5,
#         'tick_value': 0.25,
#         'minimum_fluctuation': 1.25,
#         'spread': 0.25,
#         'commission': 2.50,
#         'rolling_months': [3, 6, 9, 12],
#         'subscription_type': 'Future',
#         'subscription_details': Futures.Indices.MicroSP500EMini,
#         'resolution': Resolution.Daily
#     },
#     'USDMXN': {
#         'subscription_type': 'Forex',
#         'subscription_details': 'USDMXN',
#         'resolution': Resolution.Daily,
#         'multiplier': 1,

#     }
# }



In [11]:
# Initialize QuantBook
qb = QuantBook()

# Add Subscriptions for Futures and Forex
subscriptions = {}
for symbol, details in instruments.items():
    if details['subscription_type'] == 'Future':
        future = qb.AddFuture(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = future.Symbol
    elif details['subscription_type'] == 'Forex':
        forex = qb.AddForex(details['subscription_details'], details['resolution'])
        subscriptions[symbol] = forex.Symbol

# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data
timeseries = {}
for symbol, sub_symbol in subscriptions.items():
    history = qb.History([sub_symbol], start_date, end_date, instruments[symbol]['resolution'])
    if 'close' in history.columns:
        if instruments[symbol]['subscription_type'] == 'Future':
            history.reset_index(level=0, drop=True, inplace=True)
        timeseries[symbol] = history[['close']]
    else:
        print(f"No 'close' column found for {symbol}")

# Clean up
del details
del end_date
del future
del forex
del history
del start_date
del sub_symbol
del symbol


## Prueba Raw Data

## Print the timeseries

In [12]:
#Print symbols and first 5 entires of historical_data
for symbol, data in timeseries.items():
    print(symbol)
    print(data.tail())


ES
                              close
symbol time                        
ES 1S1 2024-05-20 20:00:00  5331.75
       2024-05-21 20:00:00  5344.00
       2024-05-22 20:00:00  5331.00
       2024-05-23 20:00:00  5286.50
       2024-05-24 20:00:00  5320.75
MES
                               close
symbol  time                        
MES 1S1 2024-05-20 20:00:00  5332.00
        2024-05-21 20:00:00  5344.00
        2024-05-22 20:00:00  5331.50
        2024-05-23 20:00:00  5286.25
        2024-05-24 20:00:00  5321.00
ZN
                                 close
symbol time                           
ZN MD  2024-05-20 19:00:00  109.015625
       2024-05-21 19:00:00  109.234375
       2024-05-22 19:00:00  109.078125
       2024-05-23 19:00:00  108.687500
       2024-05-24 19:00:00  108.781250
ZF
                                 close
symbol time                           
ZF MD  2024-05-20 19:00:00  105.765625
       2024-05-21 19:00:00  105.890625
       2024-05-22 19:00:00  105.750000
       2

## Individual dataframe creation

In [13]:

for symbol, data in timeseries.items():
    globals()[symbol] = pd.DataFrame(data)

## Summary statistics
We can begin working with the aquired data to calculate some useful statistics, such as returns, standard deviations, skew and fat tail ratios. Use the helper function to process these statistics.

### Returns Time Series

In [31]:

# Computes returns in price points, notional value, and percentage
for symbol, price_df in timeseries.items():
    price_series = price_df['close']
    merged_df = calculate_and_merge_returns(symbol, price_series, instruments)
    timeseries[symbol] = merged_df.loc[:, ['close', 'price_return', 'currency_return', 'percentage_return']]

del price_df, price_series, merged_df


### Summary Stats for each symbol

In [34]:
summary = summarize_statistics(timeseries, trading_days_per_year)


In [35]:
print("Summary Statistics for Each Symbol:")

for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])


## Backtest buy and hold 1 contract. Return is a dataframe or time series.

In [55]:
# Added to helper.py

def calculate_rolling_costs(instrument_details, timeseries):
    if 'rolling_months' not in instrument_details:
        return 0  # No rolling months, hence no rolling costs
    
    rolling_costs = 0
    rolling_months = instrument_details['rolling_months']
    years = range(timeseries.index[0][1].year, timeseries.index[-1][1].year + 1)
    
    for year in years:
        for month in rolling_months:
            # Find the first available trading day in the rolling month
            roll_date = pd.Timestamp(year=year, month=month, day=1)
            found = False
            while roll_date.month == month:
                if roll_date.date() in timeseries.index.get_level_values(1).date:
                    cost_per_roll = instrument_details['commission'] + (instrument_details['spread'] / 2) * instrument_details['multiplier']
                    rolling_costs += cost_per_roll * 2  # Buy and Sell
                    found = True
                    break
                roll_date += pd.Timedelta(days=1)
            if not found:
                print(f"No trading day found in month: {month} of year: {year}")
    
    return rolling_costs



In [56]:
# # Rolling costs per instrument. uncomment to compute

# rolling_costs_per_instrument = {}

# for symbol, data in timeseries.items():
#     instrument_details = instruments[symbol]
#     rolling_costs = calculate_rolling_costs(instrument_details, data)
#     rolling_costs_per_instrument[symbol] = rolling_costs

# print(rolling_costs_per_instrument)


In [58]:
# Added to helper.py
def compute_fx_change(returns, timeseries):
    fx_rate = timeseries['close'].iloc[-1]  # Extract the last closing price of USDMXN
    converted_returns = returns * fx_rate
    return converted_returns

In [60]:
# Added to helper.py
def simulate_buy_and_hold(timeseries, instrument_details, fx_timeseries, num_contracts=1):
    """
    Function to simulate buy and hold strategy for a futures contract

    inputs:
    timeseries: pandas DataFrame containing the price data
    instrument_details: dictionary containing the instrument details
    fx_timeseries: pandas DataFrame containing the forex data for currency conversion
    num_contracts: number of contracts to trade

    outputs:
    net_return_base: net return in base currency
    net_return_mxn: net return in MXN
    """
    first_price = timeseries['close'].iloc[0]
    last_price = timeseries['close'].iloc[-1]

    # Calculate rolling costs
    rolling_costs = calculate_rolling_costs(instrument_details, timeseries)
    
    # Calculate net return in base currency
    net_return_base = (last_price - first_price) * instrument_details['multiplier'] * num_contracts - rolling_costs
    
    # Convert net return to MXN
    net_return_mxn = compute_fx_change(net_return_base, fx_timeseries)
    
    return net_return_base, net_return_mxn

In [36]:
buyhold_results = {}

for symbol, data in timeseries.items():
    print(f"Symbol: {symbol}, Start Date: {data.index[0][1]}, End Date: {data.index[-1][1]}")
    instrument_details = instruments[symbol]
    fx_data = timeseries['USDMXN']
    num_contracts = 1  # Specify the number of contracts as needed
    
    bnh_return, bnh_return_mxn = simulate_buy_and_hold(data, instrument_details, fx_data, num_contracts)
    buyhold_results[symbol] = {
        'bnh_return': bnh_return,
        'bnh_return_mxn': bnh_return_mxn
    }
print(buyhold_results)



# Strategy 2: Risk measurement, risk target and capital


## Contract risk in base currency

Update summary dictionary to include the daily and annual contract risk 

risk in currency terms = notional exposure * std dev

notional exposure = last closing price * multiplier

In [37]:
# Update summary with contract risks

summary = calculate_contract_risk(summary, instruments, timeseries)
print("Summary Statistics for Each Symbol with Contract Risks:")
for symbol, stats in summary.items():
    print(f"{symbol}:")
    print(summary[symbol])

In [38]:
# Compute target risk in currency
target_risk_in_usd = capital * target_risk

# Compute position size
def calculate_position_size(target_risk_in_usd, contract_risk, capital):
    return target_risk_in_usd / contract_risk

In [39]:
# Calculate position size for each symbol
position_sizes = {}
for symbol, stats in summary.items():
    position_size = calculate_position_size(target_risk_in_usd, stats['annualized_contract_risk'], capital)
    position_sizes[symbol] = position_size
# Print position sizes
print ("Position sizes to achieve target risk per instrument:")
for symbol, position_size in position_sizes.items():
    print(f"{symbol}: {position_size}")
